In [2]:
# 필요 모듈 삽입


import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt 
from tensorflow.examples.tutorials.mnist import input_data
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

In [3]:
# data loading


train_data = pd.read_csv('C:/Users/student/Desktop/머신러닝/data/digit-recognizer/train.csv')
test_data = np.array(pd.read_csv('C:/Users/student/Desktop/머신러닝/data/digit-recognizer/test.csv'))

In [4]:
# 그래프 초기화

tf.reset_default_graph()



# train data 설정

x_data = np.array(train_data.drop(['label'], axis='columns', inplace=False))
y_data = np.array(train_data['label'])

y_data = pd.get_dummies(train_data.label).values
y_data.shape

scaler=MinMaxScaler()
x_data = scaler.fit_transform(x_data)

split_num = int(x_data.shape[0]*0.8)
train_x_data = x_data[:split_num]
test_x_data = x_data[split_num:]
train_y_data = y_data[:split_num]
test_y_data = y_data[split_num:]

In [5]:
# Placeholder

X = tf.placeholder(shape = [None, 784], dtype=tf.float32)
Y = tf.placeholder(shape = [None, 10], dtype=tf.float32)
drop_rate = tf.placeholder(dtype=tf.float32)


# image data 설정

x_img = tf.reshape(X, [-1,28,28,1])

In [6]:
# CNN

L1 = tf.layers.conv2d(inputs = x_img,
                      filters = 32,
                      kernel_size = [3,3],
                      padding ="SAME",
                      strides = 1, 
                      activation = tf.nn.relu)
L1 = tf.layers.max_pooling2d(inputs = L1,
                             pool_size = [2,2],
                             padding = "SAME",
                             strides = 2)


L2 = tf.layers.conv2d(inputs = L1,
                      filters = 64,
                      kernel_size = [3,3],
                      padding ="SAME",
                      strides = 1,
                      activation = tf.nn.relu)
L2 = tf.layers.max_pooling2d(inputs = L2,
                             pool_size = [2,2],
                             padding = "SAME",
                             strides = 2)


convoled_data = tf.reshape(L2, [-1,7*7*64])


layer1 = tf.layers.dense(inputs = convoled_data,
                         units = 256,
                         activation = tf.nn.relu)
layer1 = tf.layers.dropout(inputs = layer1,
                           rate = drop_rate)
layer2 = tf.layers.dense(inputs = layer1,
                         units = 128,   
                         activation = tf.nn.relu)
layer2 = tf.layers.dropout(inputs = layer2,
                           rate = drop_rate)
layer3 = tf.layers.dense(inputs = layer2,
                         units = 512,  
                         activation = tf.nn.relu)
layer3 = tf.layers.dropout(inputs = layer2,
                           rate = drop_rate)


H = tf.layers.dense(inputs = layer3, units=10)

cost = tf.losses.softmax_cross_entropy(Y, H)

train = tf.train.AdadeltaOptimizer(learning_rate = 0.001).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [8]:
for i in range(10):
    
    train_epoch = 50
    batch_size = 100


    for step in range(train_epoch):
        num_of_iter = int(x_data.shape[0] / batch_size)

        for i in range(num_of_iter):
            batch_x = x_data[i*batch_size:(i+1)*batch_size]
            batch_y = y_data[i*batch_size:(i+1)*batch_size]

            _, cost_val = sess.run([train, cost],
                                   feed_dict={X: batch_x,
                                              Y: batch_y,
                                             drop_rate: 0.3})
    
    H_val = sess.run(H, feed_dict={X: test_x_data})
    
    if i == 0:
        H_result = H_val
    else:
        H_result = np.add(H_result,H_val)
        

NameError: name 'H_result' is not defined

In [ ]:
predict = np.argmax(H_result,1)
Y = mnist.test.labels
correct = np.equal(predict, np.argmax(Y,1))
accuracy = np.mean(correct)
print(accuracy)

In [ ]:
# 단 이러한 경우, 모델은 날라가고 우리가 저장한 H의 값만 있는 것
# prediction을 할 수가 없음
# 각 모델을 저장해야 prediction을 할 수 있고, 따라서 class를 사용해야 함

# 모듈화!
# 기능의 모듈화: 자주 사용되는 코드를 함수화시켜서 반볻 처리르 하는 것을 의미
# 객체 지향의 모듈화는 '기능+데이터' 함께 모듈화한다.
# 이것이 험수와 객체지향의 가장 큰 차이점
